In [1]:
%pwd


'd:\\Ml Dl\\Project\\Text-Summaraizer\\reasearch'

In [2]:
cd ../

d:\Ml Dl\Project\Text-Summaraizer


c:\Users\ronak\anaconda3\envs\text\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
%pwd

'd:\\Ml Dl\\Project\\Text-Summaraizer'

In [ ]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataEvaluationConfig:
    root_dir : Path
    data_path : Path
    model_path : Path
    tokenizer_path : Path
    metric_file_name : Path

In [ ]:
from src.utils.common import read_yaml
from src.utils.common import create_directories
from src.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
class ConfigurationManager:
    def __init__(self,config_filepath= CONFIG_FILE_PATH,params_filepath= PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation(self) -> DataEvaluationConfig:
        config = self.config.model_evaluation                 # Extracts only the data_ingestion part of config.yaml.
        params = self.params.model_evaluation
        create_directories([config.root_dir])               # Create data_ingestion.root directory

        model_evaluation_config = DataEvaluationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_path = config.model_path,
            tokenizer_path =config.tokenizer_path,
            metric_file_name = config.metric_file_name,
        )

        return model_evaluation_config

In [ ]:
import os
from src.logger import logger
from src.entity import ModelEvaluationConfig
import torch
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
from datasets import load_from_disk
from tqdm import tqdm
from evaluate import load
import pandas as pd

class Model_Trainer:
    def __init__(self,config: ModelEvaluationConfig):
        self.config = config
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

    def  generate_batch_sized_chunks(self,list_of_elements,batch_size):
        """split the batches into smaller batches that we can process simultaneosly
        Yield successive batched sized chunks from list_of_elements"""
        for i in range(0,len(list_of_elements),batch_size):
            yield list_of_elements[i:i+batch_size] 


    def calculate_metric(self,dataset,metric,model,tokenizer,batch_size=16,device=self.device,column_text="dialogue",column_summary="summary"):
        X_batches=list(self.generate_batch_sized_chunks(dataset[column_text],batch_size))
        y_batches=list(self.generate_batch_sized_chunks(dataset[column_summary],batch_size))

        for X_batch,y_batch in tqdm(zip(X_batches,y_batches),total=len(X_batches)):
            inputs=tokenizer(X_batch,max_length=512,truncation=True,padding=True,
                return_tensors="pt")

            summaries=model.generate(input_ids=inputs["input_ids"].to(device),attention_mask=inputs["attention_mask"].to(device),
                                            length_penalty=0.8,num_beams=8,max_length=128)


            decode_summaries=[tokenizer.decode(s,skip_special_tokens=True,clean_up_tokenization_spaces=True) for s in summaries]

            decode_summaries=[d.replace(""," ") for d in decode_summaries]

            metric.add_batch(predictions=decode_summaries,references=y_batch)

        score=metric.compute()
        return score

    def evaluate(self):
        tokenizer=AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model=AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path)

        dataset=load_from_disk(self.config.data_path)

        rouge_names=["rouge1", "rouge2", "rougeL", "rougeLsum"]

        rouge_metric = load("rouge")

        score=self.calculate_metric(dataset["test"][0:10],rouge_metric,model,tokenizer,batch_size=2,
                       device=self.device,column_text="dialogue",column_summary="summary")

        rouge_dict=dict((rn,score[rn].mid.fmeasure)for rn in rouge_names)

        df=pd.DataFrame(rouge_dict,index=["pegasus"])

        df.to_csv(self.config.metric_file_name,index=False)
    